In [ ]:
# Function to decode operation

In [1]:
%pylab inline 

import mne
from mne.datasets import spm_face
from mne.decoding import GeneralizationAcrossTime
import sys
import os.path as op
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
from scipy import stats
#Add personal functions to python path
sys.path.append('/neurospin/meg/meg_tmp/Calculation_Pedro_2014/scripts/decoding/')
#sys.path.append('/Volumes/NeuroSpin2T/Calculation_Pedro_2014/scripts/decoding/')
from fldtrp2mne_calc import fldtrp2mne_calc
from calc_ClassifyTwoCond import calc_ClassifyTwoCond
from sklearn import svm
from sklearn.cross_validation import cross_val_score, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import StratifiedKFold

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Directories
data_path = '/neurospin/meg/meg_tmp/Calculation_Pedro_2014/data/mat/'
result_path = '/neurospin/meg/meg_tmp/Calculation_Pedro_2014/data/decoding/'
#data_path = '/Volumes/NeuroSpin2T/Calculation_Pedro_2014/data/mat/'
#result_path = '/Volumes/NeuroSpin2T/Calculation_Pedro_2014/data/decoding/'

#Subjects
#subjects = ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09', 's10', 
            #'s11', 's12', 's13', 's14', 's15', 's16', 's17', 's18','s19', 's21', 's22']

subjects = ['s07']


#General parameters
baseline = (-0.5, -0.05)

downsmpDec = 100

#Decoding
trainset = 'attention'
testset = 'calculation'
decCond = ['left-right_add-sub']

params = {'baseline': baseline, 'downsmpDec': downsmpDec, 'Classification': decCond, 
          'trainset': trainset, 'testset': testset}

#Results initialization
all_scores = []
all_diagonals = []

In [3]:
for sub in subjects:
    #Load data
    fname_train = op.join(data_path, sub + '_vsa.mat') 
    fname_test = op.join(data_path, sub + '_calc.mat') 

    epoch_train,info_train = fldtrp2mne_calc(fname_train, 'data', 'vsa')
    epoch_test,info_test = fldtrp2mne_calc(fname_test, 'data', 'calc')

    #Baseline-correct & filter data
    print('Baseline-correcting data for subject: ' + sub)
    epoch_train.apply_baseline(baseline)
    epoch_test.apply_baseline(baseline)

    stop
    #Define sets index and trialinfo  
    idx_trainSet = info_train['congruency'] == 1
    idx_testSet = info_test['operator'] != 0  
    info_train = info_train[idx_trainSet]
    # Correct the cue labels
    info_train[info_train['cue'] == 1] = -1
    info_train[info_train['cue'] == 2] = 1
    info_test = info_test[idx_testSet]
    labels_train = 'cue'
    labels_test = 'operator'
    
    #idx_testSet = idx_trainSet
    #info_test = info_train
    #labels_test = labels_train
    
    # Decoding train
    X_train = epoch_train[idx_trainSet] #
    y_train = np.array(info_train[labels_train]) #
    y_train = y_train.astype(numpy.float64)
    # Downsampling for decoding
    X_train.decimate(downsmpDec)
    #params.update({'testingTime':np.array(X_train.times)}) #Define test time
    # Correct the above line by something like this:
    # trainTime = {'start': 0, 'stop': 0.833, 'step': 0.833, 'length': 0.833}
    
    # Decoding test
    X_test = epoch_test[idx_testSet]#
    y_test = np.array(info_test[labels_test]) #
    y_test = y_test.astype(numpy.float64)
    # Downsampling for decoding
    X_test.decimate(downsmpDec)
    #params.update({'testingTime':np.array(X_test.times)}) #Define test time

    
    print('Decoding subject: ' + sub)
    gat, score, diagonal = calc_ClassifyTwoCond(X_train, y_train, X_test, y_test, params)
    gat.plot()
    #gat.plot_diagonal()  # plot decoding across time (correspond to GAT diagonal)

    #Store scores of different subjects in the same list
    all_scores.append(score)
    all_diagonals.append(diagonal)
    
#Transform into a numpy array   
all_scores = np.array(all_scores)
all_diagonals = np.array(all_diagonals)

# Save individual results
fname = op.join(result_path, 'Classification_ ' + '_Trainset_' + trainset + '_Testset_' + testset) 
np.save(fname, all_scores)

# Compute group averages
group_scores = np.mean(all_scores, 0)
sem_group_scores = stats.sem(all_scores, 0)

group_diagonal = np.mean(all_diagonals, 0)
sem_group_diagonal = stats.sem(all_diagonals, 0)
    

Baseline-correcting data for subject: s07
Applying baseline correction ... (mode: mean)
Applying baseline correction ... (mode: mean)


NameError: name 'stop' is not defined

In [4]:
epoch_test

<EpochsArray  |  n_events : 420 (all good), tmin : -0.5 (s), tmax : 4.5 (s), baseline : (-0.5, -0.05)>

In [5]:
info_test

,run,operand1,operator,operand2,preResult,delay,corrResult,deviant,absdeviant,rt,respSide
0,1,5,0,33,4,0,5,-1,1,1210,1
1,1,4,1,0,4,1,4,0,0,765,-1
2,1,5,-1,3,6,1,2,4,4,892,1
3,1,3,1,1,4,0,4,0,0,855,-1
4,1,5,-1,3,6,1,2,4,4,951,1
5,1,3,1,2,4,1,5,-1,1,799,1
6,1,5,0,33,4,0,5,-1,1,1125,1
7,1,6,1,3,9,0,9,0,0,930,-1
8,1,5,0,33,6,0,5,1,1,1041,1
9,1,5,-1,0,4,1,5,-1,1,766,1


In [ ]:
# Plotting  

# Plot GAT
plt.imshow(group_scores, origin = 'lower', extent = [X_train.times[0], X_train.times[len(X_train.times)-1], 
                                                     X_train.times[0], X_train.times[len(X_train.times)-1]]) #flip the matrix around
plt.axvline(0, color = 'k') #mark stimulus onset
plt.axhline(0, color = 'k') #mark stimulus onset
plt.colorbar()
plt.xlabel('Testing Time (s)')
plt.ylabel('Training Time (s)')
plt.title('Group average generalization across time \n Classification: ' + decCond[0] + ' vs ' 
+ decCond[1] + '\n Trainset: ' + trainset + ', Testset: ' + testset)



In [ ]:
# Plot Diagonal
plt.plot(X_train.times, group_diagonal, label = "Classification score")
plt.axvline(0, color = 'k') #mark stimulus onset
plt.axvline(.8, color = 'k') #mark stimulus onset
plt.axvline(1.6, color = 'k') #mark stimulus onset
plt.axvline(2.4, color = 'k') #mark stimulus onset
plt.axvline(3.2, color = 'k') #mark stimulus onset
plt.axvline(3.6, color = 'k') #mark stimulus onset

plt.axhline(0.25, color = 'k', ls = '--', label = "Chance") #mark chance level
plt.legend(loc = 'upper right')
plt.xlabel('Time (s)')
plt.ylabel('Classification Score (%)')
plt.title('Cl assification: ' + decCond[0] + ' vs ' 
+ decCond[1] + '\n Trainset: ' + trainset + ', Testset: ' + testset)


In [ ]:
group_scores.shape

In [ ]:
a = 'a'

In [ ]:
a == 'a'